In [27]:
import numpy as np

### MAP@K

In [100]:
def apk(actuals, preds, k):
    actuals = set(actuals)
    if len(preds) > k:
        preds = preds[:k]
    total_score = 0
    hits = 0
    for i, pred in enumerate(preds):
        if pred in actuals and preds not in preds[:i]:
            hits += 1
            total_score += hits / (i + 1)
    if len(actuals) == 0:
        return 0
    
    return total_score / min(k, len(actuals))

Compare tutorial: https://medium.com/@pds.bangalore/mean-average-precision-abd77d0b9a7e

In [192]:
actuals = list(range(4))
preds = [0, 100, 101, 1, 0, 3]
for k in range(1, 10):
    print(k, apk(actuals, preds, k))

1 1.0
2 0.5
3 0.3333333333333333
4 0.375
5 0.525
6 0.6916666666666667
7 0.6916666666666667
8 0.6916666666666667
9 0.6916666666666667


Order of correct cases does not matter

In [193]:
actuals = list(range(4))
preds1 = [0, 100, 101, 1, 2, 3]
preds2 = [3, 100, 101, 2, 0, 1]
for k in range(1, 9):
    print(k, apk(actuals, preds1, k), apk(actuals, preds2, k))

1 1.0 1.0
2 0.5 0.5
3 0.3333333333333333 0.3333333333333333
4 0.375 0.375
5 0.525 0.525
6 0.6916666666666667 0.6916666666666667
7 0.6916666666666667 0.6916666666666667
8 0.6916666666666667 0.6916666666666667


What if not all actuals show up in preds?

In [198]:
actuals = list(range(5))
preds = [0, 100, 101, 1, 2, 3]
for k in range(1, 9):
    print(k, apk(actuals, preds, k))

1 1.0
2 0.5
3 0.3333333333333333
4 0.375
5 0.42000000000000004
6 0.5533333333333333
7 0.5533333333333333
8 0.5533333333333333


More actuals (10), not all of them showing up:

In [199]:
actuals = list(range(10))
preds = [3, 100, 101, 1, 0, 2]
for k in range(1, 13):
    print(k, apk(actuals, preds, k))

1 1.0
2 0.5
3 0.3333333333333333
4 0.375
5 0.42000000000000004
6 0.4611111111111111
7 0.3952380952380952
8 0.3458333333333333
9 0.3074074074074074
10 0.27666666666666667
11 0.27666666666666667
12 0.27666666666666667


Compare the above two: if you had picked a smaller k (e.g., 10) than number of actuals (e.g., 20), you don't punish the missing actuals beyond k.

In [201]:
actuals0 = list(range(4))
actuals1 = list(range(5))
actuals2 = list(range(10))
preds = [3, 100, 101, 1, 0, 2]
for k in range(1, 13):
    print(k, apk(actuals0, preds, k), apk(actuals1, preds, k), apk(actuals2, preds, k))

1 1.0 1.0 1.0
2 0.5 0.5 0.5
3 0.3333333333333333 0.3333333333333333 0.3333333333333333
4 0.375 0.375 0.375
5 0.525 0.42000000000000004 0.42000000000000004
6 0.6916666666666667 0.5533333333333333 0.4611111111111111
7 0.6916666666666667 0.5533333333333333 0.3952380952380952
8 0.6916666666666667 0.5533333333333333 0.3458333333333333
9 0.6916666666666667 0.5533333333333333 0.3074074074074074
10 0.6916666666666667 0.5533333333333333 0.27666666666666667
11 0.6916666666666667 0.5533333333333333 0.27666666666666667
12 0.6916666666666667 0.5533333333333333 0.27666666666666667


### AP

In [220]:
def ap(actuals, preds, debug=False):
    actuals = set(actuals)
    num_actuals = len(actuals)
    recall_at_k = 0
    precision_at_k = 0
    hits = 0
    precisions = []
    for i, pred in enumerate(preds):
        k = i + 1
        if pred in actuals and pred not in preds[:i]:
            hit = True
            hits += 1
        else:
            hit = False
        
        recall_at_k = hits / num_actuals
        precision_at_k = hits / k
        
        if hit:
            precisions.append(precision_at_k)
        
        if debug: print('R@K: %.2f P@K: %.2f - %s' % (recall_at_k, precision_at_k, hit) )
        
        if recall_at_k == 1:
            break
            
    if recall_at_k != 1:
        raise ValueError("Recall did not reach 1 after all predictions have been checked!")
    
    return np.mean(precisions)

Compare to tutorial: https://ils.unc.edu/courses/2013_spring/inls509_001/lectures/10-EvaluationMetrics.pdf

In [221]:
actuals = list(range(10))
preds = [0, 100, 1, 2, 3, 4, 5, 101, 6, 102, 7, 103, 104, 8, 105, 106, 107, 108, 109, 9]
ap(actuals, preds, debug=True)

R@K: 0.10 P@K: 1.00 - True
R@K: 0.10 P@K: 0.50 - False
R@K: 0.20 P@K: 0.67 - True
R@K: 0.30 P@K: 0.75 - True
R@K: 0.40 P@K: 0.80 - True
R@K: 0.50 P@K: 0.83 - True
R@K: 0.60 P@K: 0.86 - True
R@K: 0.60 P@K: 0.75 - False
R@K: 0.70 P@K: 0.78 - True
R@K: 0.70 P@K: 0.70 - False
R@K: 0.80 P@K: 0.73 - True
R@K: 0.80 P@K: 0.67 - False
R@K: 0.80 P@K: 0.62 - False
R@K: 0.90 P@K: 0.64 - True
R@K: 0.90 P@K: 0.60 - False
R@K: 0.90 P@K: 0.56 - False
R@K: 0.90 P@K: 0.53 - False
R@K: 0.90 P@K: 0.50 - False
R@K: 0.90 P@K: 0.47 - False
R@K: 1.00 P@K: 0.50 - True


0.7555050505050506

Raise an exception if not all actuals are in prediction.

In [222]:
actuals = list(range(10))
preds = [0, 100, 1, 2, 3, 4, 5, 101, 6, 102, 7, 103, 104, 8, 105]
ap(actuals, preds, debug=True)

R@K: 0.10 P@K: 1.00 - True
R@K: 0.10 P@K: 0.50 - False
R@K: 0.20 P@K: 0.67 - True
R@K: 0.30 P@K: 0.75 - True
R@K: 0.40 P@K: 0.80 - True
R@K: 0.50 P@K: 0.83 - True
R@K: 0.60 P@K: 0.86 - True
R@K: 0.60 P@K: 0.75 - False
R@K: 0.70 P@K: 0.78 - True
R@K: 0.70 P@K: 0.70 - False
R@K: 0.80 P@K: 0.73 - True
R@K: 0.80 P@K: 0.67 - False
R@K: 0.80 P@K: 0.62 - False
R@K: 0.90 P@K: 0.64 - True
R@K: 0.90 P@K: 0.60 - False


ValueError: Recall did not reach 1 after all predictions have been checked!

### Comparing AP and APK

In the case when all actuals are in preds, APK == AP for big k

In [234]:
actuals = list(range(10))
preds = [0, 100, 1, 2, 3, 4, 5, 101, 6, 102, 7, 103, 104, 8, 105, 106, 107, 108, 109, 9]
for k in range(1, 23):
    rslt = apk(actuals, preds, k)
    print(k, rslt)

1 1.0
2 0.5
3 0.5555555555555555
4 0.6041666666666666
5 0.6433333333333333
6 0.6749999999999999
7 0.7010204081632653
8 0.6133928571428571
9 0.6316578483245149
10 0.5684920634920634
11 0.6412193362193361
12 0.6412193362193361
13 0.6412193362193361
14 0.7055050505050505
15 0.7055050505050505
16 0.7055050505050505
17 0.7055050505050505
18 0.7055050505050505
19 0.7055050505050505
20 0.7555050505050505
21 0.7555050505050505
22 0.7555050505050505


In [235]:
ap(actuals, preds)

0.7555050505050506

Comparison of more cases

In [238]:
actuals = list(range(9))
preds = [0, 100, 1, 2, 3, 4, 5, 101, 6, 102, 7, 103, 104, 8, 105]
apk(actuals, preds, 100), ap(actuals, preds)

(0.7838945005611673, 0.7838945005611673)

In [239]:
actuals = list(range(8))
preds = [0, 100, 1, 2, 3, 4, 5, 101, 6, 102, 7, 103, 104, 8]
apk(actuals, preds, 100), ap(actuals, preds)

(0.8015241702741702, 0.8015241702741703)

In [240]:
actuals = list(range(7))
preds = [0, 100, 1, 2, 3, 4, 5, 101, 6, 102, 7, 103]
apk(actuals, preds, 100), ap(actuals, preds)

(0.8121315192743763, 0.8121315192743763)

Reference:
- https://ils.unc.edu/courses/2013_spring/inls509_001/lectures/10-EvaluationMetrics.pdf